<a href="https://colab.research.google.com/github/NicoleBahamonMartinez/Quiz3CienciaDatosAplicada2022/blob/main/Quiz3AppliedDataScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/datascience-uniandes/linear-regression-tutorial/blob/master/forestfires.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# MINE-4101: Applied Data Science  

# Quiz 3 - Regresión

Nicole Bahamón Martínez 201629594


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

import matplotlib.pyplot as plt
import seaborn as sns

from pandas_profiling import ProfileReport

### Reading the dataset

In [ ]:
forest_df = pd.read_csv('/content/insurance.csv', sep = ',')

In [ ]:
forest_df.shape

(1338, 7)

In [ ]:
forest_df.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

In [ ]:
forest_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
forest_df = forest_df.drop_duplicates()

Arreglando variables categoricas

In [ ]:
df_one = pd.get_dummies(forest_df["sex"]) 
  
medicalcost_df = pd.concat((df_one, forest_df), axis=1) 
medicalcost_df = medicalcost_df.drop(["sex"], axis=1) 
medicalcost_df = medicalcost_df.drop(["male"], axis=1) 
medicalcost_df = medicalcost_df.rename(columns={"female": "Gender"}) 
print(medicalcost_df) 

      Gender  age     bmi  children smoker     region      charges
0          1   19  27.900         0    yes  southwest  16884.92400
1          0   18  33.770         1     no  southeast   1725.55230
2          0   28  33.000         3     no  southeast   4449.46200
3          0   33  22.705         0     no  northwest  21984.47061
4          0   32  28.880         0     no  northwest   3866.85520
...      ...  ...     ...       ...    ...        ...          ...
1333       0   50  30.970         3     no  northwest  10600.54830
1334       1   18  31.920         0     no  northeast   2205.98080
1335       1   18  36.850         0     no  southeast   1629.83350
1336       1   21  25.800         0     no  southwest   2007.94500
1337       1   61  29.070         0    yes  northwest  29141.36030

[1337 rows x 7 columns]


In [ ]:
df_one = pd.get_dummies(forest_df["smoker"]) 
  
medicalcost_df = pd.concat((df_one, medicalcost_df), axis=1) 
medicalcost_df = medicalcost_df.drop(["smoker"], axis=1) 
medicalcost_df = medicalcost_df.drop(["no"], axis=1) 
medicalcost_df = medicalcost_df.rename(columns={"yes": "Smoker"}) 
print(medicalcost_df) 

      Smoker  Gender  age     bmi  children     region      charges
0          1       1   19  27.900         0  southwest  16884.92400
1          0       0   18  33.770         1  southeast   1725.55230
2          0       0   28  33.000         3  southeast   4449.46200
3          0       0   33  22.705         0  northwest  21984.47061
4          0       0   32  28.880         0  northwest   3866.85520
...      ...     ...  ...     ...       ...        ...          ...
1333       0       0   50  30.970         3  northwest  10600.54830
1334       0       1   18  31.920         0  northeast   2205.98080
1335       0       1   18  36.850         0  southeast   1629.83350
1336       0       1   21  25.800         0  southwest   2007.94500
1337       1       1   61  29.070         0  northwest  29141.36030

[1337 rows x 7 columns]


In [ ]:
from sklearn.preprocessing import OneHotEncoder

#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column 
encoder_df = pd.DataFrame(encoder.fit_transform(medicalcost_df[['region']]).toarray())

#merge one-hot encoded columns back with original DataFrame
final_df = medicalcost_df.join(encoder_df)

final_df.drop('team', axis=1, inplace=True)

#view final df
print(final_df)

      Smoker  Gender  age     bmi  children     region      charges    0    1  \
0          1       1   19  27.900         0  southwest  16884.92400  0.0  0.0   
1          0       0   18  33.770         1  southeast   1725.55230  0.0  0.0   
2          0       0   28  33.000         3  southeast   4449.46200  0.0  0.0   
3          0       0   33  22.705         0  northwest  21984.47061  0.0  1.0   
4          0       0   32  28.880         0  northwest   3866.85520  0.0  1.0   
...      ...     ...  ...     ...       ...        ...          ...  ...  ...   
1333       0       0   50  30.970         3  northwest  10600.54830  1.0  0.0   
1334       0       1   18  31.920         0  northeast   2205.98080  0.0  0.0   
1335       0       1   18  36.850         0  southeast   1629.83350  0.0  0.0   
1336       0       1   21  25.800         0  southwest   2007.94500  0.0  1.0   
1337       1       1   61  29.070         0  northwest  29141.36030  NaN  NaN   

        2    3  
0     0.0 

In [ ]:
from sklearn.preprocessing import OneHotEncoder

#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column 
encoder_df = pd.DataFrame(encoder.fit_transform(medicalcost_df[['region']]).toarray())

#merge one-hot encoded columns back with original DataFrame
final_df = medicalcost_df.join(encoder_df)

final_df.drop('region', axis=1, inplace=True)

final_df.columns = ['Smoker', 'Gender', 'Age', 'BMI','Children','Charges','Northeast','Northwest','Southeast','Southwest']

final_df.drop(final_df.tail(1).index,inplace=True) # drop last n rows

#view final df
print(final_df)

      Smoker  Gender  Age     BMI  Children      Charges  Northeast  \
0          1       1   19  27.900         0  16884.92400        0.0   
1          0       0   18  33.770         1   1725.55230        0.0   
2          0       0   28  33.000         3   4449.46200        0.0   
3          0       0   33  22.705         0  21984.47061        0.0   
4          0       0   32  28.880         0   3866.85520        0.0   
...      ...     ...  ...     ...       ...          ...        ...   
1332       0       1   52  44.700         3  11411.68500        0.0   
1333       0       0   50  30.970         3  10600.54830        1.0   
1334       0       1   18  31.920         0   2205.98080        0.0   
1335       0       1   18  36.850         0   1629.83350        0.0   
1336       0       1   21  25.800         0   2007.94500        0.0   

      Northwest  Southeast  Southwest  
0           0.0        0.0        1.0  
1           0.0        1.0        0.0  
2           0.0        1.0 

### Features to train the model

In [ ]:
features = ['Smoker', 'Gender', 'Age', 'BMI','Children','Northeast','Northwest','Southeast','Southwest']

In [ ]:
final_df[features + ['Charges']].corr()

,Smoker,Gender,Age,BMI,Children,Northeast,Northwest,Southeast,Southwest,Charges
Smoker,1.000000,-0.078228,-0.027940,0.004138,0.008687,0.012098,0.016430,-0.004907,-0.023411,0.786951
Gender,-0.078228,1.000000,0.018665,-0.046218,-0.017172,0.009423,-0.011535,-0.002013,0.004197,-0.059095
Age,-0.027940,0.018665,1.000000,0.109749,0.042644,0.006884,0.027419,-0.022631,-0.010804,0.297246
BMI,0.004138,-0.046218,0.109749,1.000000,0.012581,-0.031490,-0.002737,0.072810,-0.041298,0.198790
Children,0.008687,-0.017172,0.042644,0.012581,1.000000,0.002486,0.024221,-0.022408,-0.003447,0.068346
Northeast,0.012098,0.009423,0.006884,-0.031490,0.002486,1.000000,-0.320158,-0.345604,-0.320810,0.016856
Northwest,0.016430,-0.011535,0.027419,-0.002737,0.024221,-0.320158,1.000000,-0.345604,-0.320810,0.035999
Southeast,-0.004907,-0.002013,-0.022631,0.072810,-0.022408,-0.345604,-0.345604,1.000000,-0.346308,-0.020794
Southwest,-0.023411,0.004197,-0.010804,-0.041298,-0.003447,-0.320810,-0.320810,-0.346308,1.000000,-0.031240
Charges,0.786951,-0.059095,0.297246,0.198790,0.068346,0.016856,0.035999,-0.020794,-0.031240,1.000000


### Splitting train and test datasets

In [ ]:
X = final_df[features]

In [ ]:
Y = final_df['Charges']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

In [ ]:
X_train.shape

(1068, 9)

In [ ]:
X_test.shape

(268, 9)

In [ ]:
plt.figure(figsize = (28, 3))
plt.boxplot(Y_train, vert = False)
plt.show()

In [ ]:
plt.figure(figsize = (28, 3))
plt.boxplot(Y_test, vert = False)
plt.show()

### Scaling features

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Training the model

In [ ]:
regr = LinearRegression()

In [ ]:
regr.fit(X_train_scaled, Y_train)

LinearRegression()

In [ ]:
regr.coef_

array([ 9.77720932e+03, -6.24361393e+01,  3.57719028e+03,  2.07302286e+03,
        5.40295909e+02,  1.08258245e+16,  1.07967251e+16,  1.12785114e+16,
        1.09944972e+16])

In [ ]:
regr.intercept_

13529.139460981645

Los atributos más importantes parecen ser la región y el genero.

### Evaluating the model

In [ ]:
preds_train = regr.predict(X_train_scaled)
preds_test = regr.predict(X_test_scaled)

In [ ]:
mean_absolute_error(Y_train, preds_train), mean_absolute_error(Y_test, preds_test)

(4299.481058149854, 3912.26787373456)

In [ ]:
np.sqrt(mean_squared_error(Y_train, preds_train)), np.sqrt(mean_squared_error(Y_test, preds_test))

(6128.758672733442, 5742.655228217935)

El error obtenido es menor al 4000 dolares en ambas divisiones del dataset. Por lo anterior, podemos definir que el error es considerable y no existe overfitting en el modelo

### Error analysis

In [ ]:
(Y_test - preds_test).describe(percentiles = [.25, .5, .75, .95, .99])

count    2.680000e+02
mean     3.665128e-13
std      2.282700e-11
min     -4.729372e-11
25%     -2.478373e-11
50%      9.549694e-12
75%      1.637090e-11
95%      2.546585e-11
99%      2.849447e-11
max      2.910383e-11
Name: Charges, dtype: float64

In [ ]:
plt.figure(figsize = (28, 3))
plt.boxplot((Y_test - preds_test), vert = False)
plt.grid()
plt.show()

In [ ]:
worst_condition = np.where((Y_test - preds_test) > 3, True, False)

In [ ]:
worst_cases = pd.concat([
    X_test.loc[worst_condition],
    Y_test.loc[worst_condition]
], axis = 1).reset_index(drop = True)

In [ ]:
worst_cases['predictions'] = pd.Series(preds_test).loc[worst_condition].reset_index(drop = True)

In [ ]:
worst_cases.shape

(0, 12)

In [ ]:
worst_cases

,Smoker,Gender,Age,BMI,Children,Charges,Northeast,Northwest,Southeast,Southwest,Charges,predictions


No hay worst cases lo que nos indica que el modelo es bueno

# Polinomial

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
X = final_df[features]
Poly_Features = PolynomialFeatures(degree=2, include_bias = False)
X_Poly = Poly_Features.fit_transform(X)
Y = final_df['Charges']
X_train, X_test, Y_train, Y_test = train_test_split(X_Poly, Y, test_size = 0.2, random_state = 1)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
regr = LinearRegression()
regr.fit(X_train_scaled, Y_train)
regr.coef_
regr.intercept_
preds_train = regr.predict(X_train_scaled)
preds_test = regr.predict(X_test_scaled)
regr.coef_

array([-1.21403396e+16, -8.45137252e+13, -9.91523367e+13,  4.02473026e+14,
        5.21330882e+14,  4.78221893e+14,  2.17791531e+14,  4.18068260e+14,
        8.56350078e+14,  7.29087222e+14, -5.07500000e+01,  2.99750000e+02,
        1.86516250e+04, -5.53250000e+02,  6.26953583e+15,  6.26953583e+15,
        6.52254702e+15,  5.94795875e+15,  4.12945482e+14, -9.17296875e+02,
        3.88750000e+01,  6.38359375e+01, -2.12618259e+14, -2.10426642e+14,
       -2.23025002e+14, -2.22358282e+14,  4.00140039e+03,  8.03365234e+02,
        2.74335938e+01,  1.26509694e+14,  1.29241208e+14,  1.35179251e+14,
        1.29973502e+14, -3.43725000e+03, -3.54500000e+02, -8.84734550e+14,
       -8.95923500e+14, -9.63063834e+14, -8.96453626e+14, -5.90500000e+02,
       -3.20150066e+14, -3.18442487e+14, -3.47375523e+14, -3.45036540e+14,
        1.84969713e+14,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        4.43617444e+14,  0.00000000e+00,  0.00000000e+00,  2.72855011e+14,
        0.00000000e+00, -

In [ ]:
mean_absolute_error(Y_train, preds_train), mean_absolute_error(Y_test, preds_test)

(2944.4405371206194, 2692.0533520560016)

In [ ]:
np.sqrt(mean_squared_error(Y_train, preds_train)), np.sqrt(mean_squared_error(Y_test, preds_test))

(4841.272424250981, 4470.885984773835)

In [ ]:
X = final_df[features]
Poly_Features = PolynomialFeatures(degree=3, include_bias = False)
X_Poly = Poly_Features.fit_transform(X)
Y = final_df['Charges']
X_train, X_test, Y_train, Y_test = train_test_split(X_Poly, Y, test_size = 0.2, random_state = 1)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
regr = LinearRegression()
regr.fit(X_train_scaled, Y_train)
regr.coef_
regr.intercept_
preds_train = regr.predict(X_train_scaled)
preds_test = regr.predict(X_test_scaled)

In [ ]:
mean_absolute_error(Y_train, preds_train), mean_absolute_error(Y_test, preds_test)

(2876.5057079479, 2959.3776459469536)

In [ ]:
np.sqrt(mean_squared_error(Y_train, preds_train)), np.sqrt(mean_squared_error(Y_test, preds_test))

(4675.408725683473, 4592.241853783964)

La transformación polineal con 2 grados reduce el error.

# Regularización

In [ ]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.1,solver="cholesky")

X = final_df[features]
Y = final_df['Charges']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
ridge_reg.fit(X_train_scaled, Y_train)
ridge_reg.coef_
ridge_reg.intercept_
preds_train = ridge_reg.predict(X_train_scaled)
preds_test = ridge_reg.predict(X_test_scaled)

In [ ]:
mean_absolute_error(Y_train, preds_train), mean_absolute_error(Y_test, preds_test)

(4299.532866070816, 3912.4864174868003)

In [ ]:
np.sqrt(mean_squared_error(Y_train, preds_train)), np.sqrt(mean_squared_error(Y_test, preds_test))

(6128.736082096302, 5742.505671152627)

In [ ]:
ridge_reg = Ridge(alpha=0.2,solver="cholesky")

X = final_df[features]
Y = final_df['Charges']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
ridge_reg.fit(X_train_scaled, Y_train)
ridge_reg.coef_
ridge_reg.intercept_
preds_train = ridge_reg.predict(X_train_scaled)
preds_test = ridge_reg.predict(X_test_scaled)

In [ ]:
mean_absolute_error(Y_train, preds_train), mean_absolute_error(Y_test, preds_test)

(4299.668402791632, 3912.5476838897544)

In [ ]:
np.sqrt(mean_squared_error(Y_train, preds_train)), np.sqrt(mean_squared_error(Y_test, preds_test))

(6128.736326212157, 5742.474967530049)

In [ ]:
ridge_reg = Ridge(alpha=0.2,solver="cholesky")

X = final_df[features]
Y = final_df['Charges']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
ridge_reg.fit(X_train_scaled, Y_train)
ridge_reg.coef_
ridge_reg.intercept_
preds_train = ridge_reg.predict(X_train_scaled)
preds_test = ridge_reg.predict(X_test_scaled)

In [ ]:
mean_absolute_error(Y_train, preds_train), mean_absolute_error(Y_test, preds_test)

(4248.785807516914, 4202.440160714766)

In [ ]:
np.sqrt(mean_squared_error(Y_train, preds_train)), np.sqrt(mean_squared_error(Y_test, preds_test))

(6009.219845682146, 6155.81224269807)

La regularización no mejora el modelo con ningún alfa